In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
warehouse_input = widgets.IntText(
    value=1,
    description='Warehouse Number:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='175px', margin='0px 5px 0px 0px')
)

user_id_input = widgets.IntText(
    value=1,
    description='User ID:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100px')
)

supplier_id_input = widgets.IntText(
    value=1,
    description='Supplier ID:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='120px')
)

In [12]:
#Local Statistics Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_profit():
    print("Showing profit...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+ f"warehouse/{warehouse_number}/profit", json = userVal)
    racks_data = response.json()
    df = pd.DataFrame(racks_data)
    display(df)

def show_top_expensive_racks():
    print("Showing top 5 expensive racks...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+ f"warehouse/{warehouse_number}/rack/expensive", json = userVal)
    racks_data = response.json()
    df = pd.DataFrame(racks_data)
    display(df)

def show_bottom_parts_type():
    print("Showing bottom 3 parts type/material...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+f"warehouse/{warehouse_number}/rack/material", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_racks_under_capacity():
    print("Showing top 5 racks under 25% capacity...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+f"warehouse/{warehouse_number}/rack/lowstock", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_top_suppliers_warehouse():
    print("Showing top 3 suppliers that supplied to the warehouse...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+f"warehouse/{warehouse_number}/transaction/suppliers", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_top_users_exchanges():
    print("Showing top 3 users that receives the most exchanges...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+f"warehouse/{warehouse_number}/users/receivesmost", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_top_days_smallest_incoming():
    print("Showing top 3 days with the smallest incoming transactions' cost...")
    user_id = user_id_input.value
    warehouse_number = warehouse_input.value
    userVal = {"user_id" : user_id}
    response = requests.post(url+f"warehouse/{warehouse_number}/transaction/leastcost", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

In [13]:
# Global Statistics Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_top_warehouses_racks():
    print("Showing top 10 warehouses with the most racks...")
    gstat = "most/rack"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_top_warehouses_incoming_transactions():
    print("Showing top 5 warehouse that has the most incoming transactions...")
    gstat = "most/incoming"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_top_warehouses_delivers_exchanges():
    print("Showing top 5 warehouse that delivers the most exchanges...")
    gstat = "most/deliver"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
        
def show_top_users_transactions():
    print("Showing top 3 users that made the most transactions...")
    gstat = "most/transactions"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_top_warehouses_least_outgoing_transactions():
    print("Showing top 3 warehouses with the least outgoing transactions...")
    gstat = "least/outgoing"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
def show_top_warehouses_cities():
    print("Showing top 3 warehouses cities with the most transactions...")
    gstat = "most/city"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

In [14]:
#Transactions Data Tables
  
def show_latest_to_oldest():
    print("Showing latest to oldest transactions...")
    warehouse_number=warehouse_input.value
    gstat = f"transactions/warehouse/{warehouse_number}"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

In [15]:
#Prices Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_part_prices():
    print("Showing part prices...")
    gstat = "parts/price/all"
    response = requests.get(url+gstat)
    parts_data = response.json()
    part_ID = []
    part_Prices = []
    for part in parts_data:
        part_ID.append(part["P_Name"])
        part_Prices.append(part["P_Price"])
    plt.figure(figsize=(10, 5))
    plt.bar(part_ID, part_Prices, color='skyblue')
    plt.xlabel('Part ID')
    plt.ylabel('Price')
    plt.title('Price of Parts by Part Name')
    plt.xticks(rotation=90) 
    plt.show()

def show_parts_supplied_by_suppliers():
    print("Showing parts supplied by suppliers...")
    supplier_id = supplier_id_input.value
    gstat = f"parts/supplier/{supplier_id}"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_parts_in_warehouse():
    print("Showing parts in warehouse...")
    warehouse_number = warehouse_input.value
    gstat = f"parts/warehouse/{warehouse_number}"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

In [16]:
# Dropdown Menus

local_stats_dropdown = widgets.Dropdown(
    options=['Choose', 'Profit', 'Top 5 Expensive Racks', 'Bottom 3 Parts Type/Material', 'Top 5 Racks Under 25% Capacity', 'Top 3 Suppliers Per Warehouse', 'Top 3 Users With Exchanges', 'Top 3 Days Smallest Incoming'],
    description='Local Stats:',
)

global_stats_dropdown = widgets.Dropdown(
    options=['Choose', 'Top 10 Warehouses With Most Racks', 'Top 5 Warehouses With Most Incoming Transactions', 'Top 5 Warehouses That Delivers The Most Exchanges','Top 3 Users That Made The Most Transactions','Top 3 Warehouses With The Least Outgoing Transactions', 'Top 3 Warehouses Cities With The Most Transactions'],
    description='Global Stats:',
)

transactions_dropdown = widgets.Dropdown(
    options=['Choose', 'Latest to Oldest'],
    description='Transactions:',
)

parts_dropdown = widgets.Dropdown(
    options=['Choose', 'Part Prices', 'Parts Supplied by Suppliers', 'Parts In Warehouse'],
    description='Parts:',
)

In [17]:
# Create an output widget to display results

output = widgets.Output()

def on_local_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Profit':
                show_profit()
            elif change['new'] == 'Top 5 Expensive Racks':
                show_top_expensive_racks()
            elif change['new'] == 'Bottom 3 Parts Type/Material':
                show_bottom_parts_type()
            elif change['new'] == 'Top 5 Racks Under 25% Capacity':
                show_racks_under_capacity()
            elif change['new'] == 'Top 3 Suppliers Per Warehouse':
                show_top_suppliers_warehouse()
            elif change['new'] == 'Top 3 Users With Exchanges':
                show_top_users_exchanges()
            elif change['new'] == 'Top 3 Days Smallest Incoming':
                show_top_days_smallest_incoming()


def on_global_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Top 10 Warehouses With Most Racks':
                show_top_warehouses_racks()
            elif change['new'] == 'Top 5 Warehouses With Most Incoming Transactions':
                show_top_warehouses_incoming_transactions()
            elif change['new'] == 'Top 5 Warehouses That Delivers The Most Exchanges':
                show_top_warehouses_delivers_exchanges()
            elif change['new'] == 'Top 3 Users That Made The Most Transactions':
                show_top_users_transactions()
            elif change['new'] == 'Top 3 Warehouses With The Least Outgoing Transactions':
                show_top_warehouses_least_outgoing_transactions()
            elif change['new'] == 'Top 3 Warehouses Cities With The Most Transactions':
                show_top_warehouses_cities()

def on_transactions_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Latest to Oldest':
                show_latest_to_oldest()

def on_parts_dropdown_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Part Prices':
                show_part_prices()
            elif change['new'] == 'Parts Supplied by Suppliers':
                show_parts_supplied_by_suppliers()
            elif change['new'] == 'Parts In Warehouse':
                show_parts_in_warehouse()

In [18]:
# Register the event handlers

local_stats_dropdown.observe(on_local_stats_change, names='value')
global_stats_dropdown.observe(on_global_stats_change, names='value')
transactions_dropdown.observe(on_transactions_change, names='value')
parts_dropdown.observe(on_parts_dropdown_change, names='value')

# Layout the widgets

title_widget = widgets.HTML(
    value="<h2 style='text-align: center;'>Datavengers - Phase 3</h2>"
)
dropdowns_vbox = widgets.HBox([local_stats_dropdown, global_stats_dropdown, transactions_dropdown, parts_dropdown],layout=widgets.Layout(justify_content='center', width='100%'))
input_box = widgets.HBox([warehouse_input, user_id_input, supplier_id_input],layout=widgets.Layout(justify_content='center', width='100%'))
main_vbox = widgets.VBox([title_widget, dropdowns_vbox, input_box, output])

# Display the main layout

display(main_vbox)